## Installing Requirements

In [ ]:
! pip install torch
! pip install transformers
! pip install sentencepiece
! pip install alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210095 sha256=376dcc3f8339a83361b25362689df2c44b0a4baefdcc882587450dab923b395c
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


## Loading The Model

In [ ]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
model_name = "persiannlp/mt5-large-parsinlu-sentiment-analysis"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
def model_predict(text_a, text_b):
    features = tokenizer( [(text_a, text_b)], padding="max_length", truncation=True, return_tensors='pt')
    output = model(**features)
    logits = output[0]
    probs = torch.nn.functional.softmax(logits, dim=1).tolist()
    idx = np.argmax(np.array(probs))
    print(labels[idx], probs)

In [ ]:
def run_model(context, query, **generator_args):
    input_ids = tokenizer.encode(context + "<sep>" + query, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
#     print(output)
    return output

## Creating Dataset

In [ ]:
input_file = "/kaggle/input/parsinlu-foods-dataset/food.jsonl"

### Creating Dataframe

In [ ]:
with open(input_file) as f:
    lines = f.read().splitlines()

In [ ]:
import pandas as pd

df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [ ]:
import json

df_inter['json_element'].apply(json.loads)

df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

df_final.head(n=5)

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3,حبوبات و سویا,NaN,NaN,NaN,NaN,NaN,NaN
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1,شربت و آبمیوه,NaN,NaN,NaN,NaN,NaN,NaN
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2,تن ماهی,2,NaN,NaN,NaN,NaN,NaN
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2,گوشت مرغ,-2,-2,NaN,NaN,NaN,NaN
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1,نوشابه,NaN,NaN,-1,1,NaN,NaN


In [ ]:
df_final['sentiment'].value_counts()

-2    462
2     440
1     420
-1    381
3     184
0      30
Name: sentiment, dtype: int64

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review              1917 non-null   object
 1   sentiment           1917 non-null   object
 2   category            1917 non-null   object
 3   aspects.طعم         886 non-null    object
 4   aspects.کیفیت       553 non-null    object
 5   aspects.ارزش غذایی  89 non-null     object
 6   aspects.ارزش خرید   414 non-null    object
 7   aspects.بسته بندی   156 non-null    object
 8   aspects.ارسال       64 non-null     object
dtypes: object(9)
memory usage: 134.9+ KB


In [ ]:
df_final.describe()

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
count,1917,1917,1917,886,553,89,414,156,64
unique,1917,6,20,6,6,6,6,5,4
top,من یه مدته فقط از این محصول استفاده میکنم اما ...,-2,تن ماهی,1,1,1,1,1,-1
freq,1,462,106,269,179,39,176,62,24


In [ ]:
df_final = df_final[['review', 'sentiment']]
df_final.head(n=5)

,review,sentiment
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1


In [ ]:
random_state = 18

### Samples

In [ ]:
samples_count = 1
sample_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=samples_count, random_state=random_state))
sample_prompts

,,review,sentiment
sentiment,,,
-1,17,پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در...,-1
-2,836,از این برند انتظار بیشتری داشتم خیلی خشک و بی ...,-2
0,1015,تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم...,0
1,238,دفعه های قبل که سفارش دادم بو میداد ولی ایندفع...,1
2,499,به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی...,2
3,916,از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنه...,3


## Prompts

In [ ]:
import time

max_retries = 5

In [ ]:
label_dict = {
    "-2" : "NEGATIVE",
    "-1" : "NEGATIVE",
    "0"  : "NEUTRAL",
    "1"  : "POSITIVE",
    "2"  : "POSITIVE",
    "3"  : "OTHER",
}

In [ ]:
per_to_eng_dict = {
    "مثبت" : "POSITIVE",
    "خنثی" : "NEUTRAL",
    "منفی" : "NEGATIVE",
    "سایر" : "OTHER",
}

In [ ]:
eng_to_per_dict = {
    "POSITIVE"  : "مثبت",
    "NEUTRAL"   : "خنثی",
    "NEGATIVE"  : "منفی",
    "OTHER"     : "سایر",
}

In [ ]:
acceptable_classes = [
    "NEGATIVE",
    "NEUTRAL",
    "POSITIVE",
    "OTHER",
]

In [ ]:
context_01 = f"""
The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it. \
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses. \
"""

In [ ]:
query_01 = f"""
Review: SAMPLE_REVIEW
Sentiment:
"""

In [ ]:
context_02 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \
"""

In [ ]:
query_02 = f"""
Review: SAMPLE_REVIEW
Sentiment:
"""

In [ ]:
context_03 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \
"""

In [ ]:
query_03 = f"""
نظر: SAMPLE_REVIEW
احساس:
"""

In [ ]:
context_04 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: مثبت، خنثی، منفی، سایر. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از خنثی استفاده کن \
"""

In [ ]:
query_04 = f"""
نظر: SAMPLE_REVIEW
احساس:
"""

In [ ]:
query_05 = "نظر شما به طور کلی راجع به این خوراکی چیست؟"

## Zero-Shot Sample Test


### Prompt #01

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشن

### Prompt #02

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_02, query=query_02.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: neutral
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنه

### Prompt #03

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_03, query=query_03.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: neutral
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: neutral
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنها

### Prompt #04

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_04, query=query_04.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: neutral
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنه

### Prompt #05

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=row['review'], query=query_05)
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: neutral
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو 

## Metrics

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter
import numpy as np

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=0)
    precision = precision_score(actual, predicted, average='weighted', zero_division=0)
    recall = recall_score(actual, predicted, average='weighted', zero_division=0)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = list(set(actual + predicted))
    classes.append('All Classes')
    metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

    probability_table = pd.DataFrame(index=metrics, columns=classes)

    for class_label in classes:
        if class_label == 'All Classes':
            probability_table[class_label]["F1 Score"] = f1
            probability_table[class_label]["Precision"] = precision
            probability_table[class_label]["Recall"] = recall
        else:
            class_precision = precision_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_recall = recall_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_f1 = f1_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            probability_table[class_label]["F1 Score"] = class_f1
            probability_table[class_label]["Precision"] = class_precision
            probability_table[class_label]["Recall"] = class_recall
    probability_table.fillna(value="-", inplace = True)
    probability_table['All Classes']["Accuracy"] = accuracy
    return probability_table

## Test data

In [ ]:
test_count = 500
test_df = df_final.sample(n=test_count)
test_df

,review,sentiment
257,من همیشه تحفه میخرم چون نمکش اندازست و کیفیت گ...,2
698,مزه آب انبه رقیق شده با ته مزه هویج خراب میده ...,-2
992,من بچم خیلی دوست داره اخه توتبلیغات دیده بود!خ...,2
1606,متاسفانه قسمتی از بسته پاره شده بود و مرغ بوی ...,-1
660,جز ترکیبات شکلات همه چی داره بار اول و آخره که...,-2
...,...,...
1205,عطر لاواتزا را ندارد اما خوب با بسته بندی شکیل...,1
633,کاملا سلیقه ای هستش من اصلا طعمش رو دوست نداشتم,-2
1784,به نظرمن خرید تخم مرغ بهتراز خرید این مقدار سف...,-1
796,هم کلسیم داره، هم 40 درصد قندش کنه... قیمتش کم...,3


In [ ]:
test_df.to_csv('test_df.csv',index=True)

## Zero-Shot Metrics

### Prompt #01

In [ ]:
from alive_progress import alive_bar

zero_01_actual = []
zero_01_predicted = []
zero_01_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_01_actual.append(label_dict[row['sentiment']])
                zero_01_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_01_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_01_predicted.append('NEUTRAL')
                    else:
                        zero_01_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 14:51.7 (0.56/s)   


In [ ]:
zero_01_results = calculate_metrics(zero_01_actual, zero_01_predicted)
zero_01_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.552
F1 Score,[0.13333333333333333],[0.8023032629558542],[0.28464419475655434],[0.2967032967032967],0.49958
Precision,[0.09523809523809523],[0.6698717948717948],[1.0],[0.20930232558139536],0.761907
Recall,[0.2222222222222222],[1.0],[0.16593886462882096],[0.5094339622641509],0.552


In [ ]:
np.save('zero_01_actual', zero_01_actual)

In [ ]:
np.save('zero_01_predicted', zero_01_predicted)

In [ ]:
np.save('zero_01_responses', zero_01_responses)

In [ ]:
zero_01_results.to_csv('zero_01_results.csv',index=True)

### Prompt #02

In [ ]:
zero_02_actual = []
zero_02_predicted = []
zero_02_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_02_actual.append(label_dict[row['sentiment']])
                zero_02_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_02_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_02_predicted.append('NEUTRAL')
                    else:
                        zero_02_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 15:04.4 (0.55/s)   


In [ ]:
zero_02_results = calculate_metrics(zero_02_actual, zero_02_predicted)
zero_02_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.552
F1 Score,[0.13333333333333333],[0.8023032629558542],[0.28464419475655434],[0.2967032967032967],0.49958
Precision,[0.09523809523809523],[0.6698717948717948],[1.0],[0.20930232558139536],0.761907
Recall,[0.2222222222222222],[1.0],[0.16593886462882096],[0.5094339622641509],0.552


In [ ]:
np.save('zero_02_actual', zero_02_actual)

In [ ]:
np.save('zero_02_predicted', zero_02_predicted)

In [ ]:
np.save('zero_02_responses', zero_02_responses)

In [ ]:
zero_02_results.to_csv('zero_02_results.csv',index=True)

### Prompt #03

In [ ]:
zero_03_actual = []
zero_03_predicted = []
zero_03_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_03_actual.append(label_dict[row['sentiment']])
                zero_03_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_03_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_03_predicted.append('NEUTRAL')
                    else:
                        zero_03_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 14:59.9 (0.56/s)   


In [ ]:
zero_03_results = calculate_metrics(zero_03_actual, zero_03_predicted)
zero_03_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.552
F1 Score,[0.13333333333333333],[0.8023032629558542],[0.28464419475655434],[0.2967032967032967],0.49958
Precision,[0.09523809523809523],[0.6698717948717948],[1.0],[0.20930232558139536],0.761907
Recall,[0.2222222222222222],[1.0],[0.16593886462882096],[0.5094339622641509],0.552


In [ ]:
np.save('zero_03_actual', zero_03_actual)

In [ ]:
np.save('zero_03_predicted', zero_03_predicted)

In [ ]:
np.save('zero_03_responses', zero_03_responses)

In [ ]:
zero_03_results.to_csv('zero_03_results.csv',index=True)

### Prompt #04

In [ ]:
zero_04_actual = []
zero_04_predicted = []
zero_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_04, query=query_04.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_04_actual.append(label_dict[row['sentiment']])
                zero_04_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_04_predicted.append('NEUTRAL')
                    else:
                        zero_04_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 15:01.9 (0.55/s)   


In [ ]:
zero_04_results = calculate_metrics(zero_04_actual, zero_04_predicted)
zero_04_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.66
F1 Score,[0.10588235294117647],[0.8169761273209549],[0.8163265306122449],[0.22950819672131148],0.741607
Precision,[0.055900621118012424],[0.9166666666666666],[0.9815950920245399],[0.875],0.926493
Recall,[1.0],[0.7368421052631579],[0.6986899563318777],[0.1320754716981132],0.66


In [ ]:
np.save('zero_04_actual', zero_04_actual)

In [ ]:
np.save('zero_04_predicted', zero_04_predicted)

In [ ]:
np.save('zero_04_responses', zero_04_responses)

In [ ]:
zero_04_results.to_csv('zero_04_results.csv',index=True)

### Prompt #05

In [ ]:
zero_05_actual = []
zero_05_predicted = []
zero_05_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=row['review'], query=query_05)
                response = " ".join(response)
                zero_05_actual.append(label_dict[row['sentiment']])
                zero_05_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_05_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_05_predicted.append('NEUTRAL')
                    else:
                        zero_05_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 11:26.3 (0.73/s)   


In [ ]:
zero_05_results = calculate_metrics(zero_05_actual, zero_05_predicted)
zero_05_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.968
F1 Score,[1.0],[0.9832134292565947],[0.9780701754385964],[0.8623853211009174],0.968352
Precision,[1.0],[0.9855769230769231],[0.9823788546255506],[0.8392857142857143],0.968865
Recall,[1.0],[0.9808612440191388],[0.9737991266375546],[0.8867924528301887],0.968


In [ ]:
np.save('zero_05_actual', zero_05_actual)

In [ ]:
np.save('zero_05_predicted', zero_05_predicted)

In [ ]:
np.save('zero_05_responses', zero_05_responses)

In [ ]:
zero_05_results.to_csv('zero_05_results.csv',index=True)

## Few-Shot Sample Test

In [ ]:
def create_examples_df(sample_size=1):
    example_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(sample_size))
    example_prompts = example_prompts.sample(frac=1).reset_index(drop=True)
    return example_prompts

In [ ]:
def create_examples_prompt():
    examples_df = create_examples_df()
    result = ""
    result_persian = ""
    i = 1
    for index, row in examples_df.iterrows():
        result += f"Example {i}:\n"
        result += f"Review: {row['review']}\n"
        result += f"Sentiment: {label_dict[row['sentiment']]}\n"

        result_persian += f"مثال {i}:\n"
        result_persian += f"نظر: {row['review']}\n"
        result_persian += f"احساس: {eng_to_per_dict[label_dict[row['sentiment']]]}\n"

        i += 1

    return result, result_persian

In [ ]:
examples, examples_persian = create_examples_prompt()
print(examples)
print("============================")
print()
print(examples_persian)

Example 1:
Review: اگه از کالای مورد نظر به اندازه کافی شناخت داشته باشی خرید اینترنتی رو پیشنهاد میدم
Sentiment: NEUTRAL
Example 2:
Review: توی این اوضاع مریضی و بیماری پاره بودن روکش سلفونی خیلی بد بود
Sentiment: NEGATIVE
Example 3:
Review: همیشه از خرید گوشتهای مهیا راضی بودم ، همیشه به روز تحویل میدادم ... اینبار برخلاف همیشه تاریخ تولیدش ۴۸ ساعت قبل بود ... یکم دلخور شدم ، ولی بازش کردم بو نمی‌داد ... بیخیال شدم ، استفاده کردم ...
Sentiment: OTHER
Example 4:
Review: با پوسته و جدا کردن پوستش سخته بخصوص جاهایی که گوشت نازکی داره با پوست کنده میشه و به هدر میره
Sentiment: NEGATIVE
Example 5:
Review: بیسکوییت خوشمزه ایه، من همیشه از این نوع میگیرم. برند مینو خرمایی هم خیلی خوشمزس، کاش دیجی موجودش میکرد. حتما پیشنهاد میکنم یکبار امتحان کنید.
Sentiment: POSITIVE
Example 6:
Review: برای اولین بار این محصول رو گرفتم. با توجه به حجم و قوطی فلزی می ارزید. ممنون از دیجی کالا که این محصول رو جدا از بقیه خریدم، بسته بندی کرده بود.
Sentiment: POSITIVE


مثال 1:
نظر: اگه از کالای مورد نظر به ان

### Prompt #01

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_01 + "Examples:\n" + examples, query=query_01.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشن

### Prompt #02

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_02 + "Examples:\n" + examples, query=query_02.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشن

### Prompt #03

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_03 + "Examples:\n" + examples, query=query_03.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: mixed
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد

### Prompt #04

In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_04 + "Examples:\n" + examples_persian, query=query_04.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: neutral
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنه

### Prompt #05


In [ ]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=row['review'], query=query_05)
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: neutral
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو 

## Few-Shot Metric

### Prompt #01

In [ ]:
few_01_actual = []
few_01_predicted = []
few_01_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01 + "Examples:\n" + examples, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_01_actual.append(label_dict[row['sentiment']])
                few_01_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_01_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_01_predicted.append('NEUTRAL')
                    else:
                        few_01_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


|████████████████████████████████████████| 500/500 [100%] in 37:14.8 (0.22/s)   


In [ ]:
few_01_results = calculate_metrics(few_01_actual, few_01_predicted)
few_01_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.684
F1 Score,[0.3636363636363636],[0.753199268738574],[0.6331360946745561],[0.5192307692307693],0.666398
Precision,[1.0],[0.6094674556213018],[0.981651376146789],[0.5294117647058824],0.778471
Recall,[0.2222222222222222],[0.9856459330143541],[0.4672489082969432],[0.5094339622641509],0.684


In [ ]:
np.save('few_01_actual', few_01_actual)

In [ ]:
np.save('few_01_predicted', few_01_predicted)

In [ ]:
np.save('few_01_responses', few_01_responses)

In [ ]:
few_01_results.to_csv('few_01_results.csv',index=True)

### Prompt #02

In [ ]:
few_02_actual = []
few_02_predicted = []
few_02_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_02 + "Examples:\n" + examples, query=query_02.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_02_actual.append(label_dict[row['sentiment']])
                few_02_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_02_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_02_predicted.append('NEUTRAL')
                    else:
                        few_02_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 38:31.4 (0.22/s)   


In [ ]:
few_02_results = calculate_metrics(few_02_actual, few_02_predicted)
few_02_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.842
F1 Score,[0.3636363636363636],[0.8708971553610503],[0.8822170900692841],[0.5858585858585859],0.836737
Precision,[1.0],[0.8024193548387096],[0.9362745098039216],[0.6304347826086957],0.849051
Recall,[0.2222222222222222],[0.9521531100478469],[0.834061135371179],[0.5471698113207547],0.842


In [ ]:
np.save('few_02_actual', few_02_actual)

In [ ]:
np.save('few_02_predicted', few_02_predicted)

In [ ]:
np.save('few_02_responses', few_02_responses)

In [ ]:
few_02_results.to_csv('few_02_results.csv',index=True)

### Prompt #03

In [ ]:
few_03_actual = []
few_03_predicted = []
few_03_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_03 + "Examples:\n" + examples, query=query_03.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_03_actual.append(label_dict[row['sentiment']])
                few_03_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_03_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_03_predicted.append('NEUTRAL')
                    else:
                        few_03_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 39:08.0 (0.21/s)   


In [ ]:
few_03_results = calculate_metrics(few_03_actual, few_03_predicted)
few_03_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.76
F1 Score,[0.5],[0.8416666666666667],[0.745308310991957],[0.5333333333333333],0.758701
Precision,[1.0],[0.7453874538745388],[0.9652777777777778],[0.43902439024390244],0.818206
Recall,[0.3333333333333333],[0.9665071770334929],[0.6069868995633187],[0.6792452830188679],0.76


In [ ]:
np.save('few_03_actual', few_03_actual)

In [ ]:
np.save('few_03_predicted', few_03_predicted)

In [ ]:
np.save('few_03_responses', few_03_responses)

In [ ]:
few_03_results.to_csv('few_03_results.csv',index=True)

### Prompt #04

In [ ]:
few_04_actual = []
few_04_predicted = []
few_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_04 + "Examples:\n" + examples_persian, query=query_04.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_04_actual.append(label_dict[row['sentiment']])
                few_04_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_04_predicted.append('NEUTRAL')
                    else:
                        few_04_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 37:45.8 (0.22/s)   


In [ ]:
few_04_results = calculate_metrics(few_04_actual, few_04_predicted)
few_04_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.848
F1 Score,[0.5],[0.8900255754475703],[0.9025844930417494],[0.425531914893617],0.839521
Precision,[1.0],[0.9560439560439561],[0.8284671532846716],[0.4878048780487805],0.848772
Recall,[0.3333333333333333],[0.8325358851674641],[0.9912663755458515],[0.37735849056603776],0.848


In [ ]:
np.save('few_04_actual', few_04_actual)

In [ ]:
np.save('few_04_predicted', few_04_predicted)

In [ ]:
np.save('few_04_responses', few_04_responses)

In [ ]:
few_04_results.to_csv('few_04_results.csv',index=True)

### Prompt #05

In [ ]:
few_05_actual = []
few_05_predicted = []
few_05_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=row['review'], query= "Examples:\n" + examples + query_05)
                response = " ".join(response)
                few_05_actual.append(label_dict[row['sentiment']])
                few_05_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_05_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_05_predicted.append('NEUTRAL')
                    else:
                        few_05_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 35:34.3 (0.23/s)   


In [ ]:
few_05_results = calculate_metrics(few_05_actual, few_05_predicted)
few_05_results

,NEUTRAL,NEGATIVE,POSITIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.56
F1 Score,[0.0],[0.6809917355371901],[0.33454545454545453],[0.5045045045045045],0.491354
Precision,[0.0],[0.5202020202020202],[1.0],[0.4827586206896552],0.726617
Recall,[0.0],[0.9856459330143541],[0.20087336244541484],[0.5283018867924528],0.56


In [ ]:
np.save('few_05_actual', few_05_actual)

In [ ]:
np.save('few_05_predicted', few_05_predicted)

In [ ]:
np.save('few_05_responses', few_05_responses)

In [ ]:
few_05_results.to_csv('few_05_results.csv',index=True)

## Boula

In [ ]:
# run_model(
#     "یک فیلم ضعیف بی محتوا بدون فیلمنامه . شوخی های سخیف .",
#     "نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع فیلم  لونه زنبور چیست؟"
# )

# run_model(
#     "فیلم تا وسط فیلم یعنی دقیقا تا جایی که معلوم میشه بچه های املشی دنبال رضان خیلی خوب و جذاب پیش میره ولی دقیقا از همونجاش سکته میزنه و خلاص...",
#     "نظر شما به صورت کلی در مورد فیلم  ژن خوک چیست؟"
# )
# run_model(
#     "اصلا به هیچ عنوان علاقه نداشتم اجرای می سی سی پی نشسته میمیرد روی پرده سینما ببینم  دیالوگ های تکراری   هلیکوپتر  ماشین  آلندلون  لئون  پاپیون  آخه چرااااااااااااااا   همون حسی که توی تالار وحدت بعد از نیم ساعت به سرم اومد امشب توی سالن سینما تجربه کردم ،حس گریز از سالن....... (ノಠ益ಠ)ノ ",
#     " نظر شما در مورد صداگذاری و جلوه های صوتی فیلم  مسخره‌باز چیست؟"
# )

# run_model(
#     " گول نخورید این رنگارنگ مینو نیست برای شرکت گرجیه و متاسفانه این محصولش اصلا مزه رنگارنگی که انتظار دارید رو نمیده ",
#     " نظر شما در مورد عطر، بو، و طعم این بیسکویت و ویفر چیست؟"
# )

# run_model(
#     "در مقایسه با سایر برندهای موجود در بازار با توجه به حراجی که داشت ارزانتر ب",
#     " شما در مورد قیمت و ارزش خرید این حبوبات و سویا چیست؟"
# )

# run_model(
#     "من پسرم عاشق ایناس ولی دیگه به خاطر حفظ محیط زیست فقط زمانهایی که مجبور باشم شیر دونه ای میخرم و سعی میکنم دیگه کمتر شیر با بسته بندی تتراپک استفاده کنم ",
#     "نظر شما به صورت کلی در مورد این شیر چیست؟"
# )
